In [101]:
f = open('./data/train_ver2.csv','r')
g = open('reduced_train.csv','w')

for line in f:
    date = line[:10]
    if date in ['2016-05-28','2016-06-28','2015-05-28','2015-06-28']:
        g.write(line)


In [7]:
with open('./data/train_ver2.csv','r') as f:
    cols=f.readline().split(',')

In [8]:
cols =[s.replace('"','') for s in cols]

In [121]:
import numpy as np 
import pandas as pd
import math
from matplotlib import pyplot as plt
import seaborn as sns

In [18]:
train=pd.read_csv('./reduced_train.csv',dtype={'indrel_1mes':str,'conyuemp':str},names=cols)

In [13]:
train.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-05-28,1061260,N,ES,H,24,2012-09-17,0.0,34,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015-05-28,1061283,N,ES,H,22,2012-09-17,0.0,34,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-05-28,1061284,N,ES,H,24,2012-09-17,0.0,34,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015-05-28,1061336,N,ES,H,23,2012-09-17,0.0,34,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-05-28,1061286,N,ES,V,29,2012-09-17,0.0,34,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
train.fecha_dato.value_counts()

2016-05-28    931453
2015-06-28    632110
2015-05-28    631957
Name: fecha_dato, dtype: int64

In [16]:
train.isnull().sum()

fecha_dato                     0
ncodpers                       0
ind_empleado                5458
pais_residencia             5458
sexo                        5469
age                            0
fecha_alta                  5458
ind_nuevo                   5458
antiguedad                     0
indrel                      5458
ult_fec_cli_1t           2192082
indrel_1mes                 9467
tiprel_1mes                 9467
indresi                     5458
indext                      5458
conyuemp                 2195228
canal_entrada              12206
indfall                     5458
tipodom                     5458
cod_prov                   16803
nomprov                    16803
ind_actividad_cliente       5458
renta                     447375
segmento                   12511
ind_ahor_fin_ult1              1
ind_aval_fin_ult1              1
ind_cco_fin_ult1               1
ind_cder_fin_ult1              1
ind_cno_fin_ult1               1
ind_ctju_fin_ult1              1
ind_ctma_f

### Data cleaning

ind_nuevo - New customer Index. 1 if the customer registered in the last 6 months.

In [23]:
#ind_nuevo new customer indicator
months_active=train.loc[train["ind_nuevo"].isnull(),:].groupby("ncodpers", sort=False).size()
months_active.max()

2

In [24]:
train.loc[train["ind_nuevo"].isnull(),"ind_nuevo"]="1"

antiguedad - Customer seniority (in months)

In [26]:
train.antiguedad=pd.to_numeric(train.antiguedad,errors="coerce")

In [35]:
train.antiguedad.value_counts()

10.0     54234
21.0     49451
33.0     47214
45.0     42845
8.0      42121
12.0     41096
9.0      38828
44.0     37409
43.0     36581
20.0     36544
22.0     36363
34.0     35024
32.0     34802
46.0     33958
7.0      33138
31.0     32852
23.0     32225
19.0     29289
36.0     26590
35.0     25832
24.0     24003
18.0     21467
6.0      19474
47.0     18336
55.0     17686
30.0     17127
54.0     16210
42.0     16030
48.0     15206
11.0     14381
         ...  
237.0     2150
224.0     1961
232.0     1944
228.0     1797
229.0     1789
234.0     1783
230.0     1728
238.0     1686
241.0     1621
233.0     1543
226.0     1543
227.0     1399
236.0     1237
240.0     1146
247.0     1114
242.0      986
239.0      978
244.0      912
245.0      842
246.0      771
243.0      748
248.0      616
251.0      435
250.0      420
252.0      253
249.0      207
253.0      151
256.0      102
254.0       85
255.0       77
Name: antiguedad, dtype: int64

bad data - antiguedad=-999999

In [34]:
train.loc[train["antiguedad"]== -999999,"antiguedad"]=train.antiguedad.median()

ult_fec_cli_1t - Last date as primary customer (if he isn't at the end of the month)
conyuemp - Spouse index. 1 if the customer is spouse of an employee
most of the data are missing, so deleting those two columns

In [36]:
del train['ult_fec_cli_1t']
del train['conyuemp']

renta - Gross income of the household

In [38]:
province_income=pd.read_csv('./data/train_ver2.csv',usecols=['cod_prov','renta'],dtype={'cod_prov':str,'renta':'float32'})

In [49]:
incomes=province_income.loc[province_income.renta.notnull(),:].groupby("cod_prov").agg({'renta':{'MedianIncome':'median'}})

In [51]:
grouped=train.groupby('cod_prov').agg({'renta':lambda x: x.median(skipna=True)}).reset_index()

In [69]:
new_incomes=pd.merge(train,grouped,how="left",on="cod_prov")

In [105]:
new_incomes=new_incomes.rename(columns={"renta_y":"renta"}).sort_values("renta").sort_values("cod_prov")

In [63]:
train.sort_values("cod_prov",inplace=True)
train=train.reset_index()
new_incomes=new_incomes.reset_index()


In [108]:
train.loc[train.renta.isnull(),"renta"] = new_incomes.loc[train.renta.isnull(),"renta"].reset_index()
train.loc[train.renta.isnull(),"renta"] = train.loc[train.renta.notnull(),"renta"].median()
train.sort_values(by="fecha_dato",inplace=True)

In [100]:
for i in np.arange(len(new_incomes)):
    if math.isnan(new_incomes.renta_x):
        new_incomes.loc[i,'renta_x']=new_incomes.loc[i,'renta_y']

In [114]:
train.isnull().sum()

index                        0
fecha_dato                   0
ncodpers                     0
ind_empleado              5458
pais_residencia           5458
sexo                      5469
age                          0
fecha_alta                5458
ind_nuevo                    0
antiguedad                5458
indrel                    5458
indrel_1mes               9468
tiprel_1mes               9468
indresi                   5458
indext                    5458
canal_entrada            12207
indfall                   5458
tipodom                   5458
cod_prov                 16803
nomprov                  16803
ind_actividad_cliente     5458
renta                        0
segmento                 12512
ind_ahor_fin_ult1            0
ind_aval_fin_ult1            0
ind_cco_fin_ult1             0
ind_cder_fin_ult1            0
ind_cno_fin_ult1             0
ind_ctju_fin_ult1            0
ind_ctma_fin_ult1            0
ind_ctop_fin_ult1            0
ind_ctpp_fin_ult1            0
ind_deco

ind_nomina_ult1/ind_nom_pens_ult1

In [115]:

train.loc[train.ind_nomina_ult1.isnull(),'ind_nomina_ult1']=0
train.loc[train.ind_nom_pens_ult1.isnull(),'ind_nom_pens_ult1']=0

delete bad records(missing ind_empleado, pais_residencia, sexo ...)

In [116]:
train=train.loc[train.ind_empleado.notnull(),:].reset_index()

In [117]:
train.isnull().sum()

level_0                      0
index                        0
fecha_dato                   0
ncodpers                     0
ind_empleado                 0
pais_residencia              0
sexo                        11
age                          0
fecha_alta                   0
ind_nuevo                    0
antiguedad                   0
indrel                       0
indrel_1mes               4010
tiprel_1mes               4010
indresi                      0
indext                       0
canal_entrada             6749
indfall                      0
tipodom                      0
cod_prov                 11345
nomprov                  11345
ind_actividad_cliente        0
renta                        0
segmento                  7054
ind_ahor_fin_ult1            0
ind_aval_fin_ult1            0
ind_cco_fin_ult1             0
ind_cder_fin_ult1            0
ind_cno_fin_ult1             0
ind_ctju_fin_ult1            0
ind_ctma_fin_ult1            0
ind_ctop_fin_ult1            0
ind_ctpp

## Explore categorical fields:

In [ ]:
cols = ["sexo","indrel_1mes","tiprel_1mes","canal_entrada","cod_prov","nomprov","segmento"]
for i in cols:
    catg=train.loc[:,i].fillna(-99)
    len_unique=len(train[i].unique())
    print("Number of unique values in", i, ":", len_unique)
    if len_unique<200:
        agg_df=train[i].value_counts()
        plt.figure(figsize=(12,6))
        sns.barplot(agg_df.index,np.log1p(agg_df.values),alpha=0.8)
        plt.xlabel(i,fontsize=12)
        plt.ylabel('Log(Number of occurence)',fontsize=12)
        plt.xticks(rotation='vertical')
        plt.show()
    print()

        

('Number of unique values in', 'sexo', ':', 3)


In [ ]:
catg=train.loc[:,"sexo"].fillna(-99)